### Data Insertion

In [28]:
import pandas as pd

df_gameplay = pd.read_csv(r'data\User Gameplay data.csv', skiprows=3)
df_deposit = pd.read_csv(r'data\Deposit Data.csv', skiprows=3)
df_withdrawal = pd.read_csv(r'data\Withdrawal Data.csv', skiprows=3)

Renaming Column

In [29]:
df_gameplay.rename(columns={'User ID': 'User_ID', 'Games Played': 'Games_Played', 'Datetime': 'Timestamp'}, inplace=True)
df_deposit.rename(columns={'User Id': 'User_ID', 'Datetime': 'Timestamp', 'Amount': 'Deposit_Amount'}, inplace=True)
df_withdrawal.rename(columns={'User Id': 'User_ID', 'Datetime': 'Timestamp', 'Amount': 'Withdrawal_Amount'}, inplace=True)

In [3]:
df_gameplay.head()

,User_ID,Games_Played,Timestamp
0,851,1,01-10-2022 0:00
1,717,1,01-10-2022 0:00
2,456,1,01-10-2022 0:00
3,424,1,01-10-2022 0:00
4,845,1,01-10-2022 0:00


In [4]:
df_deposit.head()

,User_ID,Timestamp,Deposit_Amount
0,357,01-10-2022 0:03,2000
1,776,01-10-2022 0:03,2500
2,492,01-10-2022 0:06,5000
3,803,01-10-2022 0:07,5000
4,875,01-10-2022 0:09,1500


In [5]:
df_withdrawal.head()

,User_ID,Timestamp,Withdrawal_Amount
0,190,01-10-2022 0:03,5872
1,159,01-10-2022 0:16,9540
2,164,01-10-2022 0:24,815
3,946,01-10-2022 0:29,23000
4,763,01-10-2022 0:40,9473


### Data Preprocessing

Convert 'Timestamp' columns to datetime objects

In [6]:
df_gameplay['Timestamp'] = pd.to_datetime(df_gameplay['Timestamp'], format='%d-%m-%Y %H:%M')
df_deposit['Timestamp'] = pd.to_datetime(df_deposit['Timestamp'], format='%d-%m-%Y %H:%M')
df_withdrawal['Timestamp'] = pd.to_datetime(df_withdrawal['Timestamp'], format='%d-%m-%Y %H:%M')

Extract Date and Hour for Slot calculation

In [7]:
df_gameplay['Date'] = df_gameplay['Timestamp'].dt.date
df_gameplay['Hour'] = df_gameplay['Timestamp'].dt.hour
df_deposit['Date'] = df_deposit['Timestamp'].dt.date
df_deposit['Hour'] = df_deposit['Timestamp'].dt.hour
df_withdrawal['Date'] = df_withdrawal['Timestamp'].dt.date
df_withdrawal['Hour'] = df_withdrawal['Timestamp'].dt.hour

Determine Slot (S1: 12am-12pm, S2: 12pm-12am)

In [8]:
def get_slot(hour):
    return 'S1' if hour < 12 else 'S2'

In [9]:
df_gameplay['Slot'] = df_gameplay['Hour'].apply(get_slot)
df_deposit['Slot'] = df_deposit['Hour'].apply(get_slot)
df_withdrawal['Slot'] = df_withdrawal['Hour'].apply(get_slot)

Convert Date to datetime for easier filtering

In [10]:
df_gameplay['Date'] = pd.to_datetime(df_gameplay['Date'])
df_deposit['Date'] = pd.to_datetime(df_deposit['Date'])
df_withdrawal['Date'] = pd.to_datetime(df_withdrawal['Date'])

### Part A - Loyalty Point Calculation

In [11]:
def calculate_loyalty_points(df_agg):
    df_agg['Loyalty_Points'] = 0.0
    df_agg['Loyalty_Points'] += 0.01 * df_agg['Total_Deposits']
    df_agg['Loyalty_Points'] += 0.005 * df_agg['Total_Withdrawals']
    
    df_agg['deposit_withdrawal_diff'] = df_agg['Num_Deposits'] - df_agg['Num_Withdrawals']
    df_agg['Loyalty_Points'] += 0.001 * df_agg['deposit_withdrawal_diff'].apply(lambda x: max(x, 0))
    df_agg['Loyalty_Points'] += 0.2 * df_agg['Total_Games_Played']

    return df_agg

#### 1: Slot-wise Loyalty Points

In [12]:
slots_to_calculate = {
    '2nd October Slot S1': {'date': '2022-10-02', 'slot': 'S1'},
    '16th October Slot S2': {'date': '2022-10-16', 'slot': 'S2'},
    '18th October Slot S1': {'date': '2022-10-18', 'slot': 'S1'},
    '26th October Slot S2': {'date': '2022-10-26', 'slot': 'S2'}
}

a. 2nd October Slot S1

In [13]:
loyalty_points_slotwise = {}

target_date = pd.to_datetime(slots_to_calculate['2nd October Slot S1']['date']).date()
target_slot = slots_to_calculate['2nd October Slot S1']['slot']

gameplay_filtered = df_gameplay[(df_gameplay['Date'].dt.date == target_date) & (df_gameplay['Slot'] == target_slot)]
deposit_filtered = df_deposit[(df_deposit['Date'].dt.date == target_date) & (df_deposit['Slot'] == target_slot)]
withdrawal_filtered = df_withdrawal[(df_withdrawal['Date'].dt.date == target_date) & (df_withdrawal['Slot'] == target_slot)]

agg_gameplay = gameplay_filtered.groupby('User_ID').agg(
    Total_Games_Played=('Games_Played', 'sum')
).reset_index()

agg_deposit = deposit_filtered.groupby('User_ID').agg(
    Total_Deposits=('Deposit_Amount', 'sum'),
    Num_Deposits=('User_ID', 'count')
).reset_index()

agg_withdrawal = withdrawal_filtered.groupby('User_ID').agg(
    Total_Withdrawals=('Withdrawal_Amount', 'sum'),
    Num_Withdrawals=('User_ID', 'count')
).reset_index()

df_merged_slot = pd.merge(agg_gameplay, agg_deposit, on='User_ID', how='outer')
df_merged_slot = pd.merge(df_merged_slot, agg_withdrawal, on='User_ID', how='outer')
df_merged_slot.fillna(0, inplace=True)

df_loyalty_slot = calculate_loyalty_points(df_merged_slot.copy())
loyalty_points_slotwise['2nd October Slot S1'] = df_loyalty_slot[['User_ID', 'Loyalty_Points', 'Total_Games_Played', 'Total_Deposits', 'Total_Withdrawals', 'Num_Deposits', 'Num_Withdrawals']].sort_values(by='Loyalty_Points', ascending=False)

# Save the results to a CSV file
import os
os.makedirs("output", exist_ok=True)
loyalty_points_slotwise["2nd October Slot S1"].to_csv('output/2nd_October_Slot_S1.csv', index=False)

loyalty_points_slotwise["2nd October Slot S1"].head()

,User_ID,Loyalty_Points,Total_Games_Played,Total_Deposits,Total_Withdrawals,Num_Deposits,Num_Withdrawals
395,634,1478.355,0.0,0.0,295671.0,0.0,1.0
423,672,1300.000,0.0,100000.0,60000.0,1.0,1.0
349,566,1250.406,1.0,35000.0,180041.0,2.0,1.0
605,949,677.500,0.0,5500.0,124500.0,1.0,1.0
267,446,550.202,1.0,55000.0,0.0,2.0,0.0


b. 16th October Slot S2

In [14]:
loyalty_points_slotwise = {}

target_date = pd.to_datetime(slots_to_calculate['16th October Slot S2']['date']).date()
target_slot = slots_to_calculate['16th October Slot S2']['slot']

gameplay_filtered = df_gameplay[(df_gameplay['Date'].dt.date == target_date) & (df_gameplay['Slot'] == target_slot)]
deposit_filtered = df_deposit[(df_deposit['Date'].dt.date == target_date) & (df_deposit['Slot'] == target_slot)]
withdrawal_filtered = df_withdrawal[(df_withdrawal['Date'].dt.date == target_date) & (df_withdrawal['Slot'] == target_slot)]

agg_gameplay = gameplay_filtered.groupby('User_ID').agg(
    Total_Games_Played=('Games_Played', 'sum')
).reset_index()

agg_deposit = deposit_filtered.groupby('User_ID').agg(
    Total_Deposits=('Deposit_Amount', 'sum'),
    Num_Deposits=('User_ID', 'count')
).reset_index()

agg_withdrawal = withdrawal_filtered.groupby('User_ID').agg(
    Total_Withdrawals=('Withdrawal_Amount', 'sum'),
    Num_Withdrawals=('User_ID', 'count')
).reset_index()

df_merged_slot = pd.merge(agg_gameplay, agg_deposit, on='User_ID', how='outer')
df_merged_slot = pd.merge(df_merged_slot, agg_withdrawal, on='User_ID', how='outer')
df_merged_slot.fillna(0, inplace=True)

df_loyalty_slot = calculate_loyalty_points(df_merged_slot.copy()) 
loyalty_points_slotwise['16th October Slot S2'] = df_loyalty_slot[['User_ID', 'Loyalty_Points', 'Total_Games_Played', 'Total_Deposits', 'Total_Withdrawals', 'Num_Deposits', 'Num_Withdrawals']].sort_values(by='Loyalty_Points', ascending=False)

os.makedirs("output", exist_ok=True)
loyalty_points_slotwise["16th October Slot S2"].to_csv('output/16th_October_Slot_S2.csv', index=False)

loyalty_points_slotwise["16th October Slot S2"].head()

,User_ID,Loyalty_Points,Total_Games_Played,Total_Deposits,Total_Withdrawals,Num_Deposits,Num_Withdrawals
365,634,1491.555,0.0,0.0,298311.0,0.0,1.0
122,212,999.991,0.0,99999.0,0.0,1.0,0.0
51,99,980.002,0.0,98000.0,0.0,2.0,0.0
15,28,900.004,0.0,90000.0,0.0,4.0,0.0
328,566,880.203,1.0,88000.0,0.0,3.0,0.0


c. 8th October Slot S1

In [15]:
loyalty_points_slotwise = {}

target_date = pd.to_datetime(slots_to_calculate['18th October Slot S1']['date']).date()
target_slot = slots_to_calculate['18th October Slot S1']['slot']

gameplay_filtered = df_gameplay[(df_gameplay['Date'].dt.date == target_date) & (df_gameplay['Slot'] == target_slot)]
deposit_filtered = df_deposit[(df_deposit['Date'].dt.date == target_date) & (df_deposit['Slot'] == target_slot)]
withdrawal_filtered = df_withdrawal[(df_withdrawal['Date'].dt.date == target_date) & (df_withdrawal['Slot'] == target_slot)]

agg_gameplay = gameplay_filtered.groupby('User_ID').agg(
    Total_Games_Played=('Games_Played', 'sum')
).reset_index()

agg_deposit = deposit_filtered.groupby('User_ID').agg(
    Total_Deposits=('Deposit_Amount', 'sum'),
    Num_Deposits=('User_ID', 'count')
).reset_index()

agg_withdrawal = withdrawal_filtered.groupby('User_ID').agg(
    Total_Withdrawals=('Withdrawal_Amount', 'sum'),
    Num_Withdrawals=('User_ID', 'count')
).reset_index()

df_merged_slot = pd.merge(agg_gameplay, agg_deposit, on='User_ID', how='outer')
df_merged_slot = pd.merge(df_merged_slot, agg_withdrawal, on='User_ID', how='outer')
df_merged_slot.fillna(0, inplace=True)

df_loyalty_slot = calculate_loyalty_points(df_merged_slot.copy()) 
loyalty_points_slotwise['18th October Slot S1'] = df_loyalty_slot[['User_ID', 'Loyalty_Points', 'Total_Games_Played', 'Total_Deposits', 'Total_Withdrawals', 'Num_Deposits', 'Num_Withdrawals']].sort_values(by='Loyalty_Points', ascending=False)

os.makedirs("output", exist_ok=True)
loyalty_points_slotwise["18th October Slot S1"].to_csv('output/18th_October_Slot_S1.csv', index=False)

loyalty_points_slotwise["18th October Slot S1"].head()

,User_ID,Loyalty_Points,Total_Games_Played,Total_Deposits,Total_Withdrawals,Num_Deposits,Num_Withdrawals
391,634,2723.100,0.0,0.0,544620.0,0.0,2.0
122,208,1701.401,7.0,170000.0,0.0,1.0,0.0
419,673,900.801,4.0,90000.0,0.0,1.0,0.0
92,162,770.000,0.0,12000.0,130000.0,1.0,1.0
147,245,750.000,0.0,0.0,150000.0,0.0,1.0


d. 26th October Slot S2

In [16]:
loyalty_points_slotwise = {}

target_date = pd.to_datetime(slots_to_calculate['26th October Slot S2']['date']).date()
target_slot = slots_to_calculate['26th October Slot S2']['slot']

gameplay_filtered = df_gameplay[(df_gameplay['Date'].dt.date == target_date) & (df_gameplay['Slot'] == target_slot)]
deposit_filtered = df_deposit[(df_deposit['Date'].dt.date == target_date) & (df_deposit['Slot'] == target_slot)]
withdrawal_filtered = df_withdrawal[(df_withdrawal['Date'].dt.date == target_date) & (df_withdrawal['Slot'] == target_slot)]

agg_gameplay = gameplay_filtered.groupby('User_ID').agg(
    Total_Games_Played=('Games_Played', 'sum')
).reset_index()

agg_deposit = deposit_filtered.groupby('User_ID').agg(
    Total_Deposits=('Deposit_Amount', 'sum'),
    Num_Deposits=('User_ID', 'count')
).reset_index()

agg_withdrawal = withdrawal_filtered.groupby('User_ID').agg(
    Total_Withdrawals=('Withdrawal_Amount', 'sum'),
    Num_Withdrawals=('User_ID', 'count')
).reset_index()

df_merged_slot = pd.merge(agg_gameplay, agg_deposit, on='User_ID', how='outer')
df_merged_slot = pd.merge(df_merged_slot, agg_withdrawal, on='User_ID', how='outer')
df_merged_slot.fillna(0, inplace=True)

df_loyalty_slot = calculate_loyalty_points(df_merged_slot.copy()) 
loyalty_points_slotwise['26th October Slot S2'] = df_loyalty_slot[['User_ID', 'Loyalty_Points', 'Total_Games_Played', 'Total_Deposits', 'Total_Withdrawals', 'Num_Deposits', 'Num_Withdrawals']].sort_values(by='Loyalty_Points', ascending=False)

os.makedirs("output", exist_ok=True)
loyalty_points_slotwise["26th October Slot S2"].to_csv('output/26th_October_Slot_S2.csv', index=False)

loyalty_points_slotwise["26th October Slot S2"].head()

,User_ID,Loyalty_Points,Total_Games_Played,Total_Deposits,Total_Withdrawals,Num_Deposits,Num_Withdrawals
447,714,2000.001,0.0,200000.0,0.0,1.0,0.0
227,369,1501.915,1.0,50000.0,200343.0,1.0,1.0
393,634,1237.010,1.0,10000.0,227362.0,1.0,1.0
329,538,1200.403,2.0,120000.0,0.0,3.0,0.0
0,2,900.002,0.0,90000.0,0.0,2.0,0.0


#### 2. Overall Loyalty Points and Rank for October

Aggregate data for the entire month of October

In [17]:
agg_gameplay_overall = df_gameplay.groupby('User_ID').agg(
    Total_Games_Played=('Games_Played', 'sum')
).reset_index()

agg_deposit_overall = df_deposit.groupby('User_ID').agg(
    Total_Deposits=('Deposit_Amount', 'sum'),
    Num_Deposits=('User_ID', 'count')
).reset_index()

agg_withdrawal_overall = df_withdrawal.groupby('User_ID').agg(
    Total_Withdrawals=('Withdrawal_Amount', 'sum'),
    Num_Withdrawals=('User_ID', 'count')
).reset_index()

Merge aggregated dataframes for overall calculation

In [18]:
df_merged_overall = pd.merge(agg_gameplay_overall, agg_deposit_overall, on='User_ID', how='outer')
df_merged_overall = pd.merge(df_merged_overall, agg_withdrawal_overall, on='User_ID', how='outer')

In [19]:
df_merged_overall.fillna(0, inplace=True)

Calculate overall loyalty points

In [20]:
df_loyalty_overall = calculate_loyalty_points(df_merged_overall.copy())

Rank players

In [21]:
df_loyalty_overall['Rank'] = df_loyalty_overall.sort_values(
    by=['Loyalty_Points', 'Total_Games_Played'],
    ascending=[False, False]
).reset_index().index + 1

print("\nOverall Loyalty Points and Rank (Top 5):")
df_loyalty_overall[['User_ID', 'Loyalty_Points', 'Total_Games_Played', 'Rank']].sort_values(by='Rank').head(5)


Overall Loyalty Points and Rank (Top 5):


,User_ID,Loyalty_Points,Total_Games_Played,Rank
0,0,3.000,15,1
1,1,51.601,8,2
2,2,12040.475,97,3
3,3,416.004,80,4
4,4,18.501,5,5


Save overall loyalty points to CSV

In [22]:
df_loyalty_overall.to_csv('output/Overall_Loyalty_Points_October.csv', index=False)

#### 3. Average Deposit Amount

In [23]:
average_deposit_amount = df_deposit['Deposit_Amount'].mean()
print(f"Average Deposit Amount: ${average_deposit_amount:,.2f}")

Average Deposit Amount: $5,492.19


#### 4. Average Deposit Amount Per User

In [24]:
total_deposit_per_user = df_deposit.groupby('User_ID')['Deposit_Amount'].sum().reset_index()
average_deposit_per_user = total_deposit_per_user['Deposit_Amount'].mean()
print(f"Average Deposit Amount Per User: ${average_deposit_per_user:,.2f}")

Average Deposit Amount Per User: $104,669.65


#### 5. Average Number of Games Played Per User

In [25]:
total_games_per_user = df_gameplay.groupby('User_ID')['Games_Played'].sum().reset_index()
average_games_per_user = total_games_per_user['Games_Played'].mean()
print(f"Average Number of Games Played Per User: {average_games_per_user:.2f}")

Average Number of Games Played Per User: 355.27


### Part B: How much bonus should be allocated to leaderboard players?

In [26]:
loyalty_points = {
    user_id: (
        (0.01 * df_deposit[df_deposit['User_ID'] == user_id]['Deposit_Amount'].sum()) +
        (0.005 * df_withdrawal[df_withdrawal['User_ID'] == user_id]['Withdrawal_Amount'].sum()) +
        (0.001 * max(df_deposit[df_deposit['User_ID'] == user_id].shape[0] - df_withdrawal[df_withdrawal['User_ID'] == user_id].shape[0], 0)) +
        (0.2 * df_gameplay[df_gameplay['User_ID'] == user_id]['Games_Played'].sum())
    )
    for user_id in set(df_gameplay['User_ID']).union(df_deposit['User_ID'], df_withdrawal['User_ID'])
}

In [ ]:
# Create DataFrame for loyalty points
loyalty_df = pd.DataFrame(list(loyalty_points.items()), columns=['User_ID', 'Loyalty_Points'])

# Identify top 50 players by loyalty points
top_50 = loyalty_df.nlargest(50, 'Loyalty_Points')

# Calculate total loyalty points and total games played for top 50 players
total_loyalty_points = top_50['Loyalty_Points'].sum()
total_games_played = df_gameplay[df_gameplay['User_ID'].isin(top_50['User_ID'])]['Games_Played'].sum()

# Bonus pool allocation
total_bonus_pool = 50000
loyalty_weight = 0.7
games_weight = 0.3

top_50['Points_Bonus'] = (top_50['Loyalty_Points'] / total_loyalty_points) * total_bonus_pool * loyalty_weight
top_50['Games_Bonus'] = (top_50['Loyalty_Points'] / total_games_played) * total_bonus_pool * games_weight

# Calculate total bonus for each player
top_50['Total_Bonus'] = top_50['Points_Bonus'] + top_50['Games_Bonus']

# Display the top players with their bonus allocations
top_50[['User_ID','Loyalty_Points', 'Total_Bonus']]

,User_ID,Loyalty_Points,Total_Bonus
634,634,83843.325,47772.048916
99,99,23665.737,13484.206949
672,672,22757.780,12966.873384
212,212,22199.282,12648.653731
740,740,19211.824,10946.467067
566,566,19153.755,10913.380651
714,714,16764.234,9551.885099
421,421,15446.460,8801.046985
369,369,14438.444,8226.702043
30,30,14053.375,8007.298350


In [ ]:
# Save the top 50 players with their bonus allocations to a CSV file
top_50.to_csv('output/Top_50_Players_Bonus.csv', index=False)